### instantiating services

In [ ]:

from entities.homebase.bytes import wrapperAbi, daoAbiGlobal,tokenAbiGlobal

In [2]:
from entities.homebase.entities import ProposalStatus, Proposal, StateInContract, Txaction, Token, Member, Org, Vote
from entities.homebase.bytes import wrapperAbi, daoAbiGlobal,tokenAbiGlobal
from datetime import datetime
from firebase_admin import initialize_app
from firebase_admin import firestore,credentials
from web3 import Web3
import re
from entities.project import Project
from entities.economy import Economy
import concurrent.futures

cred = credentials.Certificate('trustless.json')
initialize_app(cred)
db = firestore.client()
networks=db.collection("networks")
ceva=networks.document("Etherlink-Testnet").get()
wrapper_address = ceva.to_dict()['wrapper']
print("wrapper address :" + str(wrapper_address))
rpc="https://node.ghostnet.etherlink.com"
web3 = Web3(Web3.HTTPProvider(rpc))
projects = []
if web3.is_connected():
    print("node connected")
else:
    print("node connection failed!")

wabi=re.sub(r'\n+', ' ', wrapperAbi).strip()
wrapper_contract = web3.eth.contract(address=wrapper_address, abi=wabi)
daos_collection=db.collection('daosEtherlink-Testnet')

# native_project_abi=re.sub(r'\n+', ' ', nativeProjectAbi).strip()
# project_contract = web3.eth.contract(address=economy_address, abi=native_project_abi)

wrapper address :0xA3C99359F376bE32ADE1C259Faa01e119B038436
node connected


In [3]:
wrapper_address

'0xA3C99359F376bE32ADE1C259Faa01e119B038436'

In [ ]:
import time
counter=0
processed_transactions = set()
heartbeat=0
print("started listening...")
while True:
    latest=web3.eth.block_number,
    first=latest-4
    heartbeat+=1
    logs = web3.eth.get_logs({
        "fromBlock": first,
        "toBlock": web3.eth.block_number,
        "address": latest,
        "topics": [
            web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256)").hex(),
            None,  # Match any dao address
        ]
    })
    
    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            print("already did this one")
            continue  # Skip duplicate
        print("new dap detected !")
        decoded_event = wrapper_contract.events.NewDaoCreated().process_log(log)
        processed_transactions.add(tx_hash)
        name = decoded_event['args']['name']
        org:Org=Org(name=name)
        org.creationDate=datetime.now()
        org.address = decoded_event['args']['dao']
        org.description = decoded_event['args']['description']
        org.quorum=1
        org.votingDelay=1
        org.votingDuration=1
        org.executionDelay=decoded_event['args']['executionDelay']
        daos_collection.document(org.address).set(org.toJson())
        processed_transactions.add(tx_hash)
    if heartbeat%10==0:
        print("heartbeat: "+str(heartbeat))

    time.sleep(3)
    

started listening...


KeyboardInterrupt: 

In [12]:
logs = web3.eth.get_logs({
    "fromBlock": 13871168,
    "toBlock": 13871170,
    "address": wrapper_address,
    "topics": [
        web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256)").hex(),
        None,  # Match any dao address
    ]
})
len(logs)

1

In [11]:
web3.eth.block_number

13871699

In [9]:
decoded_event = wrapper_contract.events.NewDaoCreated().process_log(logs[0])

In [10]:
decoded_event

AttributeDict({'args': AttributeDict({'dao': '0x833365797370B3fE7F5B86E10FA62195e74b102e',
  'token': '0x69d7aF22AdDECA38b47686237c5a3d6Ce915111E',
  'initialMembers': ['0x36fE1912506E4b3B85e5a4e102237Ea81da1C3d3'],
  'initialAmounts': [100000, 1, 1, 1, 1],
  'name': 'aced',
  'symbol': 'ADS',
  'description': 'this is the description',
  'executionDelay': 90}),
 'event': 'NewDaoCreated',
 'logIndex': 11,
 'transactionIndex': 1,
 'transactionHash': HexBytes('0x36e1197c733317bed02a9cf6e9bb084ba8ccc5a04cdff9072c51a7fd9c46530c'),
 'address': '0xA3C99359F376bE32ADE1C259Faa01e119B038436',
 'blockHash': HexBytes('0x9570374a8a889a9c65db2b6c54d313caebfbbbac6ec02c14ad45089199c5f5f8'),
 'blockNumber': 13871169})

In [20]:
project_addresses = [doc.id for doc in projects_collection.stream()]
project_addresses

['0x379080EE8941CE5Ae5560d5E32A4CEBBaC7C6b92']

In [10]:
a=[1,2,3,4,5,6,7,8,9]
a[-2]

8

In [26]:

def listen_to_project_events(contract, contract_address):
    global processed_transactions
    
    # Define the topics for all events
    event_signatures = {
        "SetParties": web3.keccak(text="SetParties(address,address,string)").hex(),
        "SendFunds": web3.keccak(text="SendFunds(address,uint256)").hex(),
        "ContractorPaid": web3.keccak(text="ContractorPaid(address,uint256)").hex(),
        "ContributorWithdrawn": web3.keccak(text="ContributorWithdrawn(address,uint256)").hex(),
        "ProjectDisputed": web3.keccak(text="ProjectDisputed(address)").hex(),
        "ProjectClosed": web3.keccak(text="ProjectClosed(address)").hex(),
        "ContractSigned": web3.keccak(text="ContractSigned(address)").hex(),
        "ArbitrationDecision": web3.keccak(text="ArbitrationDecision(address,uint256,string)").hex(),
    }

    # Get logs for the current contract
    logs = web3.eth.get_logs({
        "fromBlock": web3.eth.block_number-3,
        "toBlock": web3.eth.block_number,
        "address": contract_address,
    })

    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            print("already did this one")
            continue  # Skip already processed transactions

        try:
            event_type = None
            for event_name, topic_hash in event_signatures.items():
                print("got one!")
                if log["topics"][0].hex() == topic_hash:
                    event_type = event_name
                    break

            if not event_type:
                print("Not an avent type")
                continue  # Skip logs that don't match any defined events

            # Decode and process the event
            decoded_event = getattr(contract.events, event_type)().process_log(log)
            event_data = getattr(ProjectEvents, event_type)(decoded_event)

            print(f"Event detected in contract {contract_address} ({event_type}):", event_data)

        except Exception as e:
            print(f"Error processing log for contract {contract_address}: {e}")

        # Mark the transaction as processed
        processed_transactions.add(tx_hash)


In [ ]:
while True:
    heartbeat += 1

    for address in project_addresses:
        # Get contract instance
        contract = web3.eth.contract(address=address, abi=native_project_abi)
        try:
            listen_to_project_events(contract, address)
        except Exception as e:
            
            print("Errrorrr "+e)

    if heartbeat % 10 == 0:
        print(f"Heartbeat: {heartbeat}")

    time.sleep(3)

In [28]:
import entities.services

In [ ]:
import requests
dorg_homebase_channel_id = "804689936388325376"

def send_discord_message(msg, channel_id):
    global discord_bot_token
    url = "https://discordapp.com/api/channels/"+channel_id+"/messages"
    headers = {
        "Authorization": "Bot " + discord_bot_token
    }
    body = {
        "content": msg
    }
    print("before sending "+msg)
    response = requests.request("POST", url, headers=headers, data=body)
    return {"data": str("Status code "+str(response.status_code))}

In [31]:
send_discord_message(channel_id=dorg_homebase_channel_id, msg="INDEXER IS DOWN ON TRUSTLESS!")

before sending INDEXER IS DOWN ON TRUSTLESS!


{'data': 'Status code 200'}

In [33]:
from dotenv import dotenv_values
env_variables = dotenv_values(".env")

In [ ]:
my_variable = env_variables["discord_bot_token"]
print(my_variable)